In [22]:
import sagemaker

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name

bucket = sess.default_bucket()
prefix = 'module_4/part_7'

print(role)
print(sess)
print(region)
print(bucket)
print(prefix)

arn:aws:iam::467432373215:role/service-role/AmazonSageMaker-ExecutionRole-20221206T164397
eu-west-1
sagemaker-eu-west-1-467432373215
module_4/part_7


In [23]:
import uuid

checkpoint_s3_prefix = f"{prefix}/checkpoints/{str(uuid.uuid4())}"
checkpoint_s3_uri = f"s3://{bucket}/{checkpoint_s3_prefix}/"

print(checkpoint_s3_prefix, checkpoint_s3_uri)

module_4/part_7/checkpoints/139a4165-aa81-41a2-ac14-8e6d70f6e0a7 s3://sagemaker-eu-west-1-467432373215/module_4/part_7/checkpoints/139a4165-aa81-41a2-ac14-8e6d70f6e0a7/


In [24]:
from sagemaker.tensorflow import TensorFlow

In [25]:
# --------------------------------------------------------------------------------
# 'trainingJobName' msut satisfy regular expression pattern: ^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,62}
# --------------------------------------------------------------------------------
base_job_name = "fashion-mnist-tensorflow-2"
hyperparameters = {
    "epochs": 2, 
    "batch-size": 64
}

estimator = TensorFlow(
    entry_point="fashion_mnist_tensoflow.py",
    source_dir="tf_code",
    enable_sagemaker_metrics=True,
    hyperparameters=hyperparameters,
    role=role,
    input_mode='File',
    framework_version="2.1",
    py_version="py3",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    base_job_name=base_job_name,
    checkpoint_s3_uri=checkpoint_s3_uri,
    model_dir=False
)
estimator.fit()

2022-12-12 21:24:56 Starting - Starting the training job...
2022-12-12 21:25:12 Starting - Preparing the instances for trainingProfilerReport-1670880296: InProgress
......
2022-12-12 21:26:21 Downloading - Downloading input data
2022-12-12 21:26:21 Training - Downloading the training image...
2022-12-12 21:26:48 Training - Training image download completed. Training in progress.2022-12-12 21:26:46,947 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-12-12 21:26:46,955 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-12 21:26:47,119 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-12 21:26:47,137 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-12 21:26:47,153 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-12 21:26:47,164 sagemaker-training-toolkit INFO     Invoking use

###  Descargando y usando el modelo localmente

In [26]:
estimator.model_data

's3://sagemaker-eu-west-1-467432373215/fashion-mnist-tensorflow-2-2022-12-12-21-24-55-580/output/model.tar.gz'

In [9]:
!mkdir ./fashion_mnist -p
!aws s3 cp {estimator.model_data} ./fashion_mnist/model.tar.gz
!tar -xzf fashion_mnist/model.tar.gz -C ./fashion_mnist/

download: s3://sagemaker-eu-west-1-467432373215/fashion-mnist-tensorflow-2022-12-12-20-14-55-373/output/model.tar.gz to fashion_mnist/model.tar.gz
tar: fashion_mnist/model: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [19]:
import tensorflow as tf
my_model=tf.keras.models.load_model('./fashion_mnist/00000000')

In [20]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 10)                5130      
                                                                 
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
